This is notebook gives a quick overview of this WaveNet implementation, i.e. creating the model and the data set, training the model and generating samples from it.

In [18]:
!python -m pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [1]:
import torch
from wavenet_model import *
from audio_data import WavenetDataset #WavenetDataset() uses librosa
from wavenet_training import *
from model_logging import *
import tensorflow as tf

2025-05-01 14:16:06.176581: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Model
This is an implementation of WaveNet as it was described in the original paper (https://arxiv.org/abs/1609.03499). Each layer looks like this:

```
            |----------------------------------------|      *residual*
            |                                        |
            |    |-- conv -- tanh --|                |
 -> dilate -|----|                  * ----|-- 1x1 -- + -->  *input*
                 |-- conv -- sigm --|     |
                                         1x1
                                          |
 ---------------------------------------> + ------------->  *skip*
```

Each layer dilates the input by a factor of two. After each block the dilation is reset and start from one. You can define the number of layers in each block (``layers``) and the number of blocks (``blocks``). The blocks are followed by two 1x1 convolutions and a softmax output function.
Because of the dilation operation, the independent output for multiple successive samples can be calculated efficiently. With ``output_length``, you can define the number these outputs. Empirically, it seems that a large number of skip channels is required.

In [2]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

In [3]:
model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512, 
                     output_length=16, # is this how many predictions we make?
                     dtype=dtype, 
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

* * * * * * * * * * * *
in block 0
in layer 0
receptive field:  2
in layer 1
receptive field:  4
in layer 2
receptive field:  8
in layer 3
receptive field:  16
in layer 4
receptive field:  32
in layer 5
receptive field:  64
in layer 6
receptive field:  128
in layer 7
receptive field:  256
in layer 8
receptive field:  512
in layer 9
receptive field:  1024
* * * * * * * * * * * *
in block 1
in layer 0
receptive field:  1025
in layer 1
receptive field:  1027
in layer 2
receptive field:  1031
in layer 3
receptive field:  1039
in layer 4
receptive field:  1055
in layer 5
receptive field:  1087
in layer 6
receptive field:  1151
in layer 7
receptive field:  1279
in layer 8
receptive field:  1535
in layer 9
receptive field:  2047
* * * * * * * * * * * *
in block 2
in layer 0
receptive field:  2048
in layer 1
receptive field:  2050
in layer 2
receptive field:  2054
in layer 3
receptive field:  2062
in layer 4
receptive field:  2078
in layer 5
receptive field:  2110
in layer 6
receptive field:  

## Data Set
To create the data set, you have to specify a path to a data set file. If this file already exists it will be used, if not it will be generated. If you want to generate the data set file (a ``.npz`` file), you have to specify the directory (``file_location``) in which all the audio files you want to use are located. The attribute ``target_length`` specifies the number of successive samples are used as a target and corresponds to the output length of the model. The ``item_length`` defines the number of samples in each item of the dataset and should always be ``model.receptive_field + model.output_length - 1``.

```
          |----receptive_field----|
                                |--output_length--|
example:  | | | | | | | | | | | | | | | | | | | | |
target:                           | | | | | | | | | |  
```
To create a test set, you should define a ``test_stride``. Then each ``test_stride``th item will be assigned to the test set.

In [6]:
data = WavenetDataset(dataset_file='train_samples/temp_dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')
print(f'{data[100][0].shape}')

one hot input
the dataset has 593481 items
torch.Size([256, 3085])


## Training and Logging
This implementation supports logging with TensorBoard (you need to have TensorFlow installed). You can even generate audio samples from the current snapshot of the model during training. This will happen in a background thread on the cpu, so it will not interfere with the actual training but will be rather slow. If you don't have TensorFlow, you can use the standard logger that will print out to the console.
The trainer uses Adam as default optimizer.

In [11]:
print(f"TensorFlow Version: {tf.__version__}")
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


logger = TensorboardLogger(log_interval=200,
                           validation_interval=400,
                           generate_interval=1000,
                           generate_function=generate_and_log_samples,
                           log_dir="logs/temp_model")

# logger = Logger(log_interval=200,
#                 validation_interval=400,
#                 generate_interval=1000)

TensorFlow Version: 2.10.0
2.10.0


In [8]:
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)

print('start training...')
trainer.train(batch_size=1,
              epochs=1)

start training...
epoch 0
model() called
forward() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate() called
wavenet_dilate() called
dilate() called
dilate

KeyboardInterrupt: 

## Generating
This model has the Fast Wavenet Generation Algorithm (https://arxiv.org/abs/1611.09482) implemented. This might run faster on the cpu. You can give some starting data (of at least the length of receptive field) or let the model generate from zero. In my experience, a temperature between 0.5 and 1.0 yields the best results, but this may depend on the data set.

In [10]:
start_data = data[250000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

RuntimeError: expand(torch.FloatTensor{[32, 1]}, size=[32]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

In [ ]:
import IPython.display as ipd

ipd.Audio(generated, rate=16000)